# Langchain crash course

In [15]:
from secret import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

## LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



Tandoori Majesty


In [3]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\nMaharaja Bites'

## Prompt Templates

In [4]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.
 

Buon Appetito!


## Chains

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\nTaco Loco'

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\nTaco Terraza.'

In [9]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [10]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



-Butter Chicken
-Tandoori Chicken
-Matar Paneer
-Malai Kofta
-Vegetable Biryani
-Palak Paneer
-Aloo Gobi
-Chana Masala
-Dal Makhani
-Naan Bread
-Raita
-Kheer


#### Sequential Chain

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
chain({"cuisine": "Mexican"})

{'cuisine': 'Mexican',
 'restaurant_name': '\n\nFiesta de Mexico',
 'menu_items': '\n\n-Tacos\n-Enchiladas\n-Burritos\n-Quesadillas\n-Chimichangas\n-Fajitas\n-Tostadas\n-Tamales\n-Mole Poblano\n-Chiles Rellenos\n-Tortilla Soup\n-Nachos\n-Guacamole\n-Ceviche\n-Taco Salad\n-Fried Plantains\n-Sopapillas\n-Churros\n-Margaritas'}

## Agents

In [38]:
# make sure yoy have installed this package: pip install google-search-results
from secret import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key


#### serpapi and llm-math tool

In [39]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass  `serpapi_api_key` as a named parameter. (type=value_error)

#### Wikipedia and llm-math tool

In [27]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX;  angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$207 billion as of October 2023, according to the Bloomberg Billionaires Index, and $231 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two y

'Elon Musk was born on June 28, 1971 and is currently 52 years old in 2023.'

## Memory

In [40]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [41]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [42]:
chain.memory

In [43]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [44]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [45]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [46]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [24]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [25]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [26]:
convo.run("How much is 5+5?")

' 10.'

In [27]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [29]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [30]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [31]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [32]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."